In [2]:
import requests
import json
import igdb
import re

IGDB API documentation https://api-docs.igdb.com/#authentication
<br>Wrapper library https://github.com/twitchtv/igdb-api-python

In [ ]:
url = 'https://id.twitch.tv/oauth2/token'

client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

myobj = {"client_id":client_id,
        "client_secret":client_secret,
        "grant_type":"client_credentials"}

x = requests.post(url, data = myobj) #authentication request
y = json.loads(x.content.decode('utf-8'))

print(y) #returns access_token, expires_in, token_type(bearer)

In [10]:
from igdb.wrapper import IGDBWrapper
from igdb.igdbapi_pb2 import GameResult
wrapper = IGDBWrapper("YOUR_CLIENT_ID", "YOUR_APP_ACCESS_TOKEN")

In [4]:
import pandas as pd
import numpy as np

games = pd.read_csv('steam_coops.csv', index_col=0)
games

,game_id,title,release_date,price,rating_inPercent,url
0,1426210,It Takes Two,"25 Mar, 2021",2499.0,95.0,https://store.steampowered.com/app/1426210/It_...
1,1172470,Apex Legends™,"4 Nov, 2020",0.0,90.0,https://store.steampowered.com/app/1172470/Ape...
2,1085660,Destiny 2,"1 Oct, 2019",0.0,85.0,https://store.steampowered.com/app/1085660/Des...
3,1056640,Phantasy Star Online 2 New Genesis,"5 Aug, 2020",0.0,80.0,https://store.steampowered.com/app/1056640/Pha...
4,252490,Rust,"8 Feb, 2018",725.0,86.0,https://store.steampowered.com/app/252490/Rust...
...,...,...,...,...,...,...
382,647740,Airmen,"25 Jul, 2017",349.0,85.0,https://store.steampowered.com/app/647740/Airm...
383,982510,Dungeon Hunter 5,"21 May, 2019",0.0,49.0,https://store.steampowered.com/app/982510/Dung...
384,696140,No Heroes Here,"2 Oct, 2017",349.0,90.0,https://store.steampowered.com/app/696140/No_H...
385,1191330,Inferna,"20 Dec, 2019",0.0,62.0,https://store.steampowered.com/app/1191330/Inf...


In [12]:
def get_igdb_rating(y, kind):
    try:
        return y[0][kind]
    except KeyError:
        return None

In [17]:
igdb = []
errors = []
for i in range(games.shape[0]-370):
    name = games.loc[i, 'title']
    
    # Some names from Steam contain special characters, 
    # which do not appear in IGDB game's names, 
    # so we need to get rid of these characters.
    # Some games also have a second name after / or -,
    # but we will use only the first part of the name.
    
    new_name = re.sub(r'[@#$©®™]', '', name)
    new_name = re.sub("'","\'", new_name)
    new_name = new_name.split('/')[0]
    new_name = new_name.split('-')[0]
    
    try:
        byte_array = wrapper.api_request(
                    'games',
                    f'fields id, aggregated_rating, rating; offset 0; where name="{new_name}";'
                  )
        y = json.loads(byte_array)
        scores = {
                    'game id': games.loc[i, 'game_id'],
                    'igdb id': y[0]['id'],
                    'rating': get_igdb_rating(y, 'aggregated_rating'),
                    'user_rating': get_igdb_rating(y, 'rating'),
        }
        igdb.append(scores)
    except requests.HTTPError:
        print("HTTP ", new_name)
    except UnicodeEncodeError:
        print("Unicode ", name, ' ', new_name)
        errors.append(name)
    except IndexError:
        print("ERROR", name, ' ', new_name, ' ', byte_array)
        errors.append(name)

ERROR Soulworker   Soulworker   b'[]'


In [ ]:
errs = pd.Series(errors)
errs.to_csv('errors names.csv', index=True)

In [18]:
errors = pd.read_csv('errors names.csv', index_col=0)
errors

,0
0,Soulworker
1,Divinity: Original Sin 2 - Definitive Edition
2,封灵档案
3,Black Desert
4,Realm of the Mad God Exalt
...,...
80,GearStorm - Armored Survival
81,Reign Of Kings
82,SG/ZH: School Girl/Zombie Hunter
83,Rogue Trooper Redux


In [ ]:
igdb_ratings = pd.DataFrame(igdb)
igdb_ratings.to_csv('igdb ratings.csv', index=False)

In [11]:
igdb_r = pd.read_csv('igdb ratings.csv', index_col=0)
igdb_r

,game id,igdb id,rating,user_rating,got reviews
0,1426210.0,135243.0,90.428571,92.209693,1.0
1,1172470.0,114795.0,88.000000,78.854477,0.0
2,1085660.0,25657.0,82.440000,74.121451,1.0
3,1056640.0,136042.0,NaN,NaN,0.0
4,252490.0,3277.0,77.500000,72.535607,1.0
...,...,...,...,...,...
300,1191330.0,126774.0,NaN,NaN,0.0
301,675210.0,54502.0,NaN,50.000000,0.0
302,204300.0,1281.0,72.500000,66.089588,1.0
303,394510.0,14523.0,84.166667,73.371240,1.0


In [12]:
igdb_r[igdb_r['got reviews'] == 1].dropna()

,game id,igdb id,rating,user_rating,got reviews
0,1426210.0,135243.0,90.428571,92.209693,1.0
2,1085660.0,25657.0,82.440000,74.121451,1.0
4,252490.0,3277.0,77.500000,72.535607,1.0
5,381210.0,18866.0,67.111111,72.049621,1.0
6,892970.0,104967.0,90.000000,91.942847,1.0
9,1097150.0,119313.0,85.111111,76.231006,1.0
10,1086940.0,119171.0,75.000000,80.233371,1.0
11,594650.0,7291.0,74.800000,80.096318,1.0
12,239140.0,3042.0,72.058824,81.298383,1.0
14,945360.0,111469.0,81.750000,71.109811,1.0
